In [21]:
#import the necessary libraries 
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import seaborn as sns
import folium
import statsmodels.api as sm
import scipy.stats
import numpy as np
from math import sqrt
import statsmodels.formula.api as smf



flows = pd.read_csv("london_flows.csv")

In [22]:
flows.head(10)


,station_origin,station_destination,flows,population,jobs,distance
0,Abbey Road,Bank and Monument,0,599,78549,8131.525097
1,Abbey Road,Beckton,1,599,442,8510.121774
2,Abbey Road,Blackwall,3,599,665,3775.448872
3,Abbey Road,Canary Wharf,1,599,58772,5086.514220
4,Abbey Road,Canning Town,37,599,15428,2228.923167
5,Abbey Road,Crossharbour,1,599,1208,6686.475560
6,Abbey Road,Custom House,0,599,845,3824.855630
7,Abbey Road,Cutty Sark,2,599,1748,8503.898909
8,Abbey Road,Cyprus,7,599,850,6532.099618
9,Abbey Road,Devons Road,1,599,611,3958.324171


In [23]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from sklearn.metrics import mean_squared_error


flows = pd.read_csv("london_flows.csv")
print(flows.info())
print(flows.head())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61474 entries, 0 to 61473
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   station_origin       61474 non-null  object 
 1   station_destination  61474 non-null  object 
 2   flows                61474 non-null  int64  
 3   population           61474 non-null  int64  
 4   jobs                 61474 non-null  int64  
 5   distance             61474 non-null  float64
dtypes: float64(1), int64(3), object(2)
memory usage: 2.8+ MB
None
  station_origin station_destination  flows  population   jobs     distance
0     Abbey Road   Bank and Monument      0         599  78549  8131.525097
1     Abbey Road             Beckton      1         599    442  8510.121774
2     Abbey Road           Blackwall      3         599    665  3775.448872
3     Abbey Road        Canary Wharf      1         599  58772  5086.514220
4     Abbey Road        Canning Town     37  

In [24]:
import pandas as pd




cdatasubmat = flows.pivot_table(index='station_origin', columns='station_destination', values='flows', aggfunc='sum')


cdatasubmat.fillna(0, inplace=True)


cdatasubmat['All'] = cdatasubmat.sum(axis=1)
cdatasubmat.loc['All'] = cdatasubmat.sum()

cdatasubmat



station_destination,Abbey Road,Acton Central,Acton Town,Aldgate,Aldgate East,All Saints,Alperton,Amersham,Anerley,Angel,...,Wimbledon,Wimbledon Park,Wood Green,Wood Lane,Wood Street,Woodford,Woodgrange Park,Woodside Park,Woolwich Arsenal,All
station_origin,,,,,,,,,,,,,,,,,,,,,
Abbey Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,599.0
Acton Central,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1224.0
Acton Town,0.0,0.0,0.0,3.0,17.0,0.0,35.0,0.0,0.0,11.0,...,77.0,3.0,6.0,9.0,0.0,0.0,0.0,0.0,0.0,3745.0
Aldgate,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,...,0.0,0.0,4.0,8.0,0.0,0.0,0.0,0.0,0.0,2886.0
Aldgate East,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,...,24.0,0.0,0.0,12.0,0.0,1.0,0.0,1.0,0.0,3172.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Woodford,0.0,0.0,2.0,5.0,47.0,0.0,0.0,0.0,0.0,22.0,...,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,4868.0
Woodgrange Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,530.0
Woodside Park,0.0,0.0,1.0,26.0,11.0,0.0,0.0,0.0,0.0,59.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3093.0


In [26]:
flows['log_dest_jobs'] = np.log(flows['jobs'] + 0.001)
flows['log_dest_pop'] = np.log(flows['population'] + 0.001)

prodsim_formula_exp = 'flows ~ log_dest_jobs + log_dest_pop + distance -1' 



uncosim = smf.glm(formula = prodsim_formula_exp, data=flows, family=sm.families.Poisson()).fit()
print(uncosim.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  flows   No. Observations:                61474
Model:                            GLM   Df Residuals:                    61471
Model Family:                 Poisson   Df Model:                            2
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.6472e+06
Date:                Wed, 04 Sep 2024   Deviance:                   3.1223e+06
Time:                        17:11:22   Pearson chi2:                 6.70e+06
No. Iterations:                     6   Pseudo R-squ. (CS):              1.000
Covariance Type:            nonrobust                                         
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
log_dest_jobs     0.3442      0.000    774.969

In [27]:
O_i = pd.DataFrame(flows.groupby(["station_origin"])["flows"].agg(np.sum))
O_i.rename(columns={"flows":"O_i"}, inplace = True)
flows = flows.merge(O_i, on = "station_origin", how = "left" )

D_j = pd.DataFrame(flows.groupby(["station_destination"])["flows"].agg(np.sum))
D_j.rename(columns={"flows":"D_j"}, inplace = True)
flows = flows.merge(D_j, on = "station_destination", how = "left" )

/tmp/ipykernel_3217/3741157495.py:1: FutureWarning: The provided callable <function sum at 0x7f9ef42bdbc0> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'sum' instead.
  O_i = pd.DataFrame(flows.groupby(["station_origin"])["flows"].agg(np.sum))
/tmp/ipykernel_3217/3741157495.py:5: FutureWarning: The provided callable <function sum at 0x7f9ef42bdbc0> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'sum' instead.
  D_j = pd.DataFrame(flows.groupby(["station_destination"])["flows"].agg(np.sum))


In [28]:
flows

,station_origin,station_destination,flows,population,jobs,distance,log_dest_jobs,log_dest_pop,O_i,D_j
0,Abbey Road,Bank and Monument,0,599,78549,8131.525097,11.271478,6.395263,599,78549
1,Abbey Road,Beckton,1,599,442,8510.121774,6.091312,6.395263,599,442
2,Abbey Road,Blackwall,3,599,665,3775.448872,6.499789,6.395263,599,665
3,Abbey Road,Canary Wharf,1,599,58772,5086.514220,10.981421,6.395263,599,58772
4,Abbey Road,Canning Town,37,599,15428,2228.923167,9.643939,6.395263,599,15428
...,...,...,...,...,...,...,...,...,...,...
61469,Woolwich Arsenal,Tower Gateway,127,7892,3342,13401.795549,8.114325,8.973605,7892,3342
61470,Woolwich Arsenal,West Ham,608,7892,5487,8701.454361,8.610137,8.973605,7892,5487
61471,Woolwich Arsenal,West India Quay,6,7892,400,9536.720451,5.991467,8.973605,7892,400
61472,Woolwich Arsenal,West Silvertown,81,7892,893,5355.248554,6.794588,8.973605,7892,893


In [29]:
alpha = uncosim.params[0]
gamma = uncosim.params[1]
beta = -uncosim.params[2]

/tmp/ipykernel_3217/3657918305.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  alpha = uncosim.params[0]
/tmp/ipykernel_3217/3657918305.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  gamma = uncosim.params[1]
/tmp/ipykernel_3217/3657918305.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  beta = -uncosim.params[2]


In [33]:
flows["unconstrainedEst2"] = np.exp( alpha*flows["log_dest_jobs"] 
                                       + gamma*flows["log_dest_pop"] 
                                       - beta*flows["distance"])


In [35]:
predictions = uncosim.get_prediction()
predictions_summary_frame = predictions.summary_frame()
flows["fitted"] = predictions_summary_frame["mean"]

In [36]:
flows.head()

,station_origin,station_destination,flows,population,jobs,distance,log_dest_jobs,log_dest_pop,O_i,D_j,unconstrainedEst2,fitted
0,Abbey Road,Bank and Monument,0,599,78549,8131.525097,11.271478,6.395263,599,78549,77.324833,77.324833
1,Abbey Road,Beckton,1,599,442,8510.121774,6.091312,6.395263,599,442,12.358238,12.358238
2,Abbey Road,Blackwall,3,599,665,3775.448872,6.499789,6.395263,599,665,26.850598,26.850598
3,Abbey Road,Canary Wharf,1,599,58772,5086.514220,10.981421,6.395263,599,58772,105.300182,105.300182
4,Abbey Road,Canning Town,37,599,15428,2228.923167,9.643939,6.395263,599,15428,97.511104,97.511104


In [37]:
# define r^2 and RMSE calculations
def CalcRSquared(observed, estimated):
    """Calculate the r^2 from a series of observed and estimated target values
    inputs:
    Observed: Series of actual observed values
    estimated: Series of predicted values"""
    
    r, p = scipy.stats.pearsonr(observed, estimated)
    R2 = r **2
    
    return R2

def CalcRMSE(observed, estimated):
    """Calculate Root Mean Square Error between a series of observed and estimated values
    inputs:
    Observed: Series of actual observed values
    estimated: Series of predicted values"""
    
    res = (observed -estimated)**2
    RMSE = round(np.sqrt(res.mean()), 3)
    
    return RMSE

In [38]:
CalcRSquared(flows["flows"], flows["unconstrainedEst2"])

0.16993444506444386

In [39]:
CalcRMSE(flows["flows"], flows["unconstrainedEst2"])

120.949